In [ ]:
import cv2
import numpy as np
import sys
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input

# === Load the trained model (make sure the path is correct)
MODEL_PATH = 'best_inception_model.keras'

try:
    model = load_model(MODEL_PATH)
except Exception as e:
    print(f"Failed to load model from {MODEL_PATH}: {e}")
    sys.exit(1)

# === Preprocess input image
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found or unreadable: {image_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB (Inception expects RGB)
    img = cv2.resize(img, (299, 299))           # InceptionV3 default size
    img = preprocess_input(img)                 # Normalize using InceptionV3's method
    img = np.expand_dims(img, axis=0)           # Add batch dimension
    return img

# === Run diagnosis
def diagnose(image_path):
    try:
        img = preprocess_image(image_path)
        prediction = model.predict(img)[0][0]  # Binary prediction: tumor vs no tumor
        confidence = prediction * 100

        print(f"\n🖼️  Image: {os.path.basename(image_path)}")
        print(f"📊 Prediction Confidence: {confidence:.2f}%")

        if prediction >= 0.5:
            print("\033[91m Tumor Detected\033[0m")  # Red text
        else:
            print("\033[92m No Tumor Detected\033[0m")  # Green text

    except Exception as e:
        print(f"Error during diagnosis: {e}")

# === Main CLI interface
if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python diagnose_brain_tumor.py path_to_image.jpg")
    else:
        image_path = sys.argv[1]
        if not os.path.isfile(image_path):
            print(f"File does not exist: {image_path}")
        else:
            diagnose(image_path)
